In [36]:
import tabula
import pandas as pd
%pip install pulp
from pulp import *
lista_tabelas = tabula.read_pdf("Trabalho3-parte1.pdf",pages="1" ,guess=False,lattice=True) #ou "all"

Note: you may need to restart the kernel to use updated packages.


In [37]:
df_demanda = lista_tabelas[0]
df_capacidade = lista_tabelas[1]
df_custos = lista_tabelas[2]
df_resolucao = lista_tabelas[3]

In [38]:
display(df_demanda)
display(df_capacidade)
display(df_custos)
display(df_resolucao)

,Contratos,Demanda,Número Mínimo de Fornecedores
0,Washington,100,3
1,Oregon,65,2
2,California,100,3
3,Idaho,70,2
4,Nevada,120,3
5,Montana,60,2
6,Wyoming,75,2
7,Arizona,100,3
8,Utah,95,3
9,Colorado,85,3


,Filiais,Capacidade,Entrega Mínima
0,Seattle,175,25
1,San Francisco,175,30
2,Las Vegas,175,30
3,Phoenix,175,35
4,Denver,175,25


,Custo de Entrega (R$/unid.),Washington,Oregon,California,Idaho,Nevada,Unnamed: 0,Montana,Wyoming,Arizona,Utah,Colorado
0,Seattle,10,15,10,15,20,NaN,20,20,40,10,30
1,San Francisco,30,15,10,20,10,NaN,20,20,30,20,30
2,Las Vegas,20,10,5,15,10,NaN,15,15,10,5,5
3,Phoenix,40,25,15,20,10,NaN,30,30,10,15,10
4,Denver,30,30,25,10,5,NaN,35,35,15,5,10


,Distribuição (unid.),Washington,Oregon,California,Idaho,Nevada,Montana,Wyoming,Arizona,Utah,Colorado
0,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,San Francisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Las Vegas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Phoenix,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Denver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df_demanda = df_demanda.set_index("Contratos")
df_capacidade = df_capacidade.set_index("Filiais")
df_custos = df_custos.set_index("Custo de Entrega (R$/unid.)")
df_custos = df_custos.drop("Unnamed: 0", axis=1)#exclui a coluna repetida
df_resolucao = df_resolucao.set_index("Distribuição (unid.)")

In [40]:
display(df_demanda)
display(df_capacidade)
display(df_custos)
display(df_resolucao)

,Demanda,Número Mínimo de Fornecedores
Contratos,,
Washington,100,3
Oregon,65,2
California,100,3
Idaho,70,2
Nevada,120,3
Montana,60,2
Wyoming,75,2
Arizona,100,3
Utah,95,3


,Capacidade,Entrega Mínima
Filiais,,
Seattle,175,25
San Francisco,175,30
Las Vegas,175,30
Phoenix,175,35
Denver,175,25


,Washington,Oregon,California,Idaho,Nevada,Montana,Wyoming,Arizona,Utah,Colorado
Custo de Entrega (R$/unid.),,,,,,,,,,
Seattle,10,15,10,15,20,20,20,40,10,30
San Francisco,30,15,10,20,10,20,20,30,20,30
Las Vegas,20,10,5,15,10,15,15,10,5,5
Phoenix,40,25,15,20,10,30,30,10,15,10
Denver,30,30,25,10,5,35,35,15,5,10


,Washington,Oregon,California,Idaho,Nevada,Montana,Wyoming,Arizona,Utah,Colorado
Distribuição (unid.),,,,,,,,,,
Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
San Francisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Las Vegas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Phoenix,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Denver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
contratos = df_demanda.index
filiais = df_capacidade.index
# The cost data is made into a dictionary
costs = makeDict([filiais, contratos], df_custos.values, 0)


In [42]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("OTIMIZAÇÃO DA DISTRIBUIÇÃO DE PRODUTOS", LpMinimize)
# Creates a list of tuples containing all the possible routes for transport
Routes = [(w, b) for w in filiais for b in contratos]
# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
objective_vars = LpVariable.dicts("QTD_produtos", (filiais, contratos), 0, None, LpInteger)
# variáveis binárias, que indicam sem há ou não entrega
binary_vars = LpVariable.dicts("?Rota", (filiais, contratos), 0, None, LpInteger)
prob += (
    lpSum([objective_vars[w][b] * costs[w][b] for (w, b) in Routes]),
    "Custo_total_de_atendimento",
)

C:\ProgramData\Anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [43]:

for (f,c) in Routes: 
    prob += (binary_vars[f][c]<=1)# restrições para a binary_vars
for c in contratos:
    prob += (lpSum([binary_vars[f][c] for f in filiais]) >= df_demanda['Número Mínimo de Fornecedores'][c])
    # Restrições de demanda
    prob += (lpSum([objective_vars[f][c] for f in filiais]) == df_demanda['Demanda'][c])

for f in filiais:
    prob += (lpSum([objective_vars[f][c] for c in contratos]) <= df_capacidade['Capacidade'][f])
    for c in contratos:
        # Restrições para a capacidade de entrega
        prob += (objective_vars[f][c]<=df_capacidade['Capacidade'][f]*binary_vars[f][c])
        # Restrições para entrega mínima
        prob += (objective_vars[f][c]>=df_capacidade['Entrega Mínima'][f]*binary_vars[f][c])
    
print(prob)

OTIMIZAÇÃO_DA_DISTRIBUIÇÃO_DE_PRODUTOS:
MINIMIZE
15*QTD_produtos_Denver_Arizona + 25*QTD_produtos_Denver_California + 10*QTD_produtos_Denver_Colorado + 10*QTD_produtos_Denver_Idaho + 35*QTD_produtos_Denver_Montana + 5*QTD_produtos_Denver_Nevada + 30*QTD_produtos_Denver_Oregon + 5*QTD_produtos_Denver_Utah + 30*QTD_produtos_Denver_Washington + 35*QTD_produtos_Denver_Wyoming + 10*QTD_produtos_Las_Vegas_Arizona + 5*QTD_produtos_Las_Vegas_California + 5*QTD_produtos_Las_Vegas_Colorado + 15*QTD_produtos_Las_Vegas_Idaho + 15*QTD_produtos_Las_Vegas_Montana + 10*QTD_produtos_Las_Vegas_Nevada + 10*QTD_produtos_Las_Vegas_Oregon + 5*QTD_produtos_Las_Vegas_Utah + 20*QTD_produtos_Las_Vegas_Washington + 15*QTD_produtos_Las_Vegas_Wyoming + 10*QTD_produtos_Phoenix_Arizona + 15*QTD_produtos_Phoenix_California + 10*QTD_produtos_Phoenix_Colorado + 20*QTD_produtos_Phoenix_Idaho + 30*QTD_produtos_Phoenix_Montana + 10*QTD_produtos_Phoenix_Nevada + 25*QTD_produtos_Phoenix_Oregon + 15*QTD_produtos_Phoenix_Utah

In [44]:
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)
    string_var = str(v.name)
    list_var = string_var.split("_")
    if list_var[0] == "QTD":
        filial = list_var[2]
        contrato = list_var[3]
        if filial == 'San':
            filial = 'San Francisco'
            contrato = list_var[4]
        if filial == 'Las':
            filial = 'Las Vegas'
            contrato = list_var[4]
        df_resolucao[contrato][filial]=v.varValue
    

# The optimised objective function value is printed to the screen
print("Total Cost of Transportation = ", value(prob.objective))

Status: Optimal
?Rota_Denver_Arizona = 1.0
?Rota_Denver_California = 0.0
?Rota_Denver_Colorado = 1.0
?Rota_Denver_Idaho = 1.0
?Rota_Denver_Montana = 0.0
?Rota_Denver_Nevada = 1.0
?Rota_Denver_Oregon = 0.0
?Rota_Denver_Utah = 1.0
?Rota_Denver_Washington = 1.0
?Rota_Denver_Wyoming = 0.0
?Rota_Las_Vegas_Arizona = 1.0
?Rota_Las_Vegas_California = 1.0
?Rota_Las_Vegas_Colorado = 0.0
?Rota_Las_Vegas_Idaho = 0.0
?Rota_Las_Vegas_Montana = 0.0
?Rota_Las_Vegas_Nevada = 0.0
?Rota_Las_Vegas_Oregon = 1.0
?Rota_Las_Vegas_Utah = 1.0
?Rota_Las_Vegas_Washington = 1.0
?Rota_Las_Vegas_Wyoming = 0.0
?Rota_Phoenix_Arizona = 1.0
?Rota_Phoenix_California = 0.0
?Rota_Phoenix_Colorado = 1.0
?Rota_Phoenix_Idaho = 1.0
?Rota_Phoenix_Montana = 0.0
?Rota_Phoenix_Nevada = 1.0
?Rota_Phoenix_Oregon = 0.0
?Rota_Phoenix_Utah = 0.0
?Rota_Phoenix_Washington = 0.0
?Rota_Phoenix_Wyoming = 0.0
?Rota_San_Francisco_Arizona = 0.0
?Rota_San_Francisco_California = 1.0
?Rota_San_Francisco_Colorado = 0.0
?Rota_San_Francisco_Idaho = 

In [47]:
display(df_resolucao)

,Washington,Oregon,California,Idaho,Nevada,Montana,Wyoming,Arizona,Utah,Colorado
Distribuição (unid.),,,,,,,,,,
Seattle,45.0,0.0,25.0,0.0,0.0,30.0,25.0,0.0,25.0,25.0
San Francisco,0.0,35.0,30.0,0.0,30.0,30.0,50.0,0.0,0.0,0.0
Las Vegas,30.0,30.0,45.0,0.0,0.0,0.0,0.0,30.0,40.0,0.0
Phoenix,0.0,0.0,0.0,35.0,55.0,0.0,0.0,45.0,0.0,35.0
Denver,25.0,0.0,0.0,35.0,35.0,0.0,0.0,25.0,30.0,25.0
